In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('CM_raw.csv')

In [2]:
df = df[df['comments'] != 0]

In [4]:
import pandas as pd

# 删除 name 和 course_name 都重复的行，保留第一次出现的行
df_unique = df.drop_duplicates(subset=['name', 'course_name'])

# 定义成绩的排序顺序
grade_order = ['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-', 'F']

# 创建一个字典，将成绩映射到数值
grade_to_num = {grade: i for i, grade in enumerate(grade_order)}

# 创建一个新的列来存储数值化的成绩
df_unique['overall_grade_num'] = df_unique['overall_grade'].map(grade_to_num)

# 按 overall_grade_num 降序排序
df_sorted = df_unique.sort_values('overall_grade_num', ascending=True)

# 删除辅助列
df_sorted = df_sorted.drop('overall_grade_num', axis=1)

# 重置索引
df_sorted = df_sorted.reset_index(drop=True)

# 设置显示选项
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

# 显示去重并排序后的数据框
print(df_sorted)

# 输出处理前后的行数
print(f"\n原始数据行数: {len(df)}")
print(f"处理后数据行数: {len(df_sorted)}")

     course_name                                               name overall_grade grade difficulty useful  comments
0       HSCI4000                                          YUAN ZHEN             A     A          A      A         1
1       FINC2000                                           XIE JING             A     A          A      A         7
2       ACCT3006                                      YUEN CHUN YIP             A     A          A      A         3
3       GELH1001                                      CHEONG IM LAN             A     A          A      A        10
4       GPAD2013                                      CHENG SAM KEE             A     A          A      A         3
5       HSCI3006                        SAN MING WANG / SHAO NINGYI             A     A          A      A         3
6       MATH4001                                     LEONG IENG TAK             A     A          A      A         1
7       PORT1001        MARIO RUI LIMA DE OLIVEIRA PINHARANDA NUNES     

C:\Users\Administrator\AppData\Local\Temp\ipykernel_41928\6197830.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['overall_grade_num'] = df_unique['overall_grade'].map(grade_to_num)


In [6]:
df_ratings = df_sorted
df_ratings[:10]

,course_name,name,overall_grade,grade,difficulty,useful,comments
0,HSCI4000,YUAN ZHEN,A,A,A,A,1
1,FINC2000,XIE JING,A,A,A,A,7
2,ACCT3006,YUEN CHUN YIP,A,A,A,A,3
3,GELH1001,CHEONG IM LAN,A,A,A,A,10
4,GPAD2013,CHENG SAM KEE,A,A,A,A,3
5,HSCI3006,SAN MING WANG / SHAO NINGYI,A,A,A,A,3
6,MATH4001,LEONG IENG TAK,A,A,A,A,1
7,PORT1001,MARIO RUI LIMA DE OLIVEIRA PINHARANDA NUNES,A,A,B-,A,2
8,CPED1001,CHAU HONG CHENG,A,A,A,A,4
9,CISC3001,WU YUAN,A,A,A,A,3


In [7]:
import pandas as pd

# 读取 Excel 文件
df_schedule = pd.read_excel("CM.xlsx", skiprows=3)

# 重命名关键列
df_schedule = df_schedule.rename(columns={
    "Unnamed: 2": "Course Code",
    "Unnamed: 3": "Course Title",
    "Unnamed: 4": "Section",
    "Unnamed: 8": "Teacher Information",
    "Unnamed: 10": "Day",
    "Unnamed: 11": "Time From",
    "Unnamed: 12": "Time To"
})

# 创建时间信息
df_schedule['Time'] = df_schedule['Day'] + ' ' + df_schedule['Time From'].astype(str) + '-' + df_schedule['Time To'].astype(str)

# 创建一个映射字典
course_info = df_schedule.groupby(['Course Code', 'Teacher Information']).apply(
    lambda x: x[['Section', 'Time']].to_dict('records')
).to_dict()

# 处理 df_ratings
result = []
for _, row in df_ratings.iterrows():
    key = (row['course_name'], row['name'])
    sections_info = course_info.get(key, [])
    
    if not sections_info:
        result.append({**row.to_dict(), 'section': None, 'time': None})
    else:
        for section_info in sections_info:
            result.append({
                **row.to_dict(),
                'section': section_info['Section'],
                'time': section_info['Time']
            })

# 创建新的 DataFrame
df_expanded = pd.DataFrame(result)

# 显示结果
df_expanded

C:\Users\Administrator\AppData\Local\Temp\ipykernel_41928\1812863707.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  course_info = df_schedule.groupby(['Course Code', 'Teacher Information']).apply(


,course_name,name,overall_grade,grade,difficulty,useful,comments,section,time
0,HSCI4000,YUAN ZHEN,A,A,A,A,1,None,None
1,FINC2000,XIE JING,A,A,A,A,7,001,MON 16:00-17:15
2,FINC2000,XIE JING,A,A,A,A,7,001,THU 16:00-17:15
3,FINC2000,XIE JING,A,A,A,A,7,002,MON 17:30-18:45
4,FINC2000,XIE JING,A,A,A,A,7,002,THU 17:30-18:45
5,ACCT3006,YUEN CHUN YIP,A,A,A,A,3,001,WED 10:00-11:15
6,GELH1001,CHEONG IM LAN,A,A,A,A,10,003,MON 14:30-15:45
7,GELH1001,CHEONG IM LAN,A,A,A,A,10,003,THU 14:30-15:45
8,GELH1001,CHEONG IM LAN,A,A,A,A,10,004,MON 16:00-17:15
9,GELH1001,CHEONG IM LAN,A,A,A,A,10,004,THU 16:00-17:15


In [8]:
# 定义一个函数来合并时间
def merge_times(times):
    return ' & '.join(sorted(set(t for t in times if pd.notna(t))))

# 按照 course_name, name, section 分组，然后聚合数据
df_merged = df_expanded.groupby(['course_name', 'name', 'section']).agg({
    'overall_grade': 'first',
    'grade': 'first',
    'difficulty': 'first',
    'useful': 'first',
    'comments': 'first',
    'time': merge_times
}).reset_index()

# 对结果进行排序
df_merged = df_merged.sort_values(['course_name', 'name', 'section'])

# 重置索引
df_merged = df_merged.reset_index(drop=True)

df_merged[:10]

,course_name,name,section,overall_grade,grade,difficulty,useful,comments,time
0,ACCT1000,SAM IENG,002,D+,D,D,D+,173,MON 13:00-14:15 & THU 13:00-14:15
1,ACCT1000,SAM IENG,003,D+,D,D,D+,173,MON 14:30-15:45 & THU 14:30-15:45
2,ACCT1000,SAM IENG,004,D+,D,D,D+,173,FRI 11:30-12:45 & TUE 11:30-12:45
3,ACCT1000,SAM IENG,005,D+,D,D,D+,173,FRI 14:30-15:45 & TUE 14:30-15:45
4,ACCT1000,SAM IENG,006,D+,D,D,D+,173,FRI 16:00-17:15 & TUE 16:00-17:15
5,ACCT1000,TCHIANG VAN KAM,001,B-,B,C+,B,67,MON 08:30-09:45 & THU 08:30-09:45
6,ACCT1000,TCHIANG VAN KAM,007,B-,B,C+,B,67,MON 10:00-11:15 & THU 10:00-11:15
7,ACCT2000,TCHIANG VAN KAM,001,B,A,B-,A,22,FRI 14:30-15:45 & TUE 14:30-15:45
8,ACCT2000,TCHIANG VAN KAM,002,B,A,B-,A,22,FRI 17:30-18:45 & TUE 17:30-18:45
9,ACCT2001,BYUNG CHERL SOHN,001,C,B-,C-,C+,13,MON 16:00-17:15 & THU 16:00-17:15


In [9]:
df_merged['section'] = df_merged['section'].astype(int)

In [10]:
df = df_merged

In [11]:
import pandas as pd

# 定义一个字典来映射等级到数值，以便排序
grade_order = {
    'A+': 12, 'A': 11, 'A-': 10,
    'B+': 9, 'B': 8, 'B-': 7,
    'C+': 6, 'C': 5, 'C-': 4,
    'D+': 3, 'D': 2, 'D-': 1,
    'F': 0
}

def grade_to_numeric(grade):
    return grade_order.get(grade, -1)  # 如果等级不在字典中，返回-1

# 假设数据已经在一个名为 df 的 DataFrame 中
sorted_df = df.sort_values(
    by=['overall_grade', 'course_name', 'name'],
    key=lambda x: x.map(lambda y: grade_to_numeric(y) if x.name == 'overall_grade' else y),
    ascending=[False, True, True]  # overall_grade 降序，其他升序
)

# 显示排序后的结果
sorted_df

,course_name,name,section,overall_grade,grade,difficulty,useful,comments,time
18,ACCT3006,YUEN CHUN YIP,1,A,A,A,A,3,WED 10:00-11:15
27,APAC4005,ZHOU YINNING,1,A,A,A,A,2,MON 14:30-15:45 & THU 14:30-15:45
84,CISC3001,WU YUAN,1,A,A,A,A,3,MON 17:30-18:45 & THU 17:30-18:45 & WED 10:00-...
110,COMM1000,MA SIYUAN,3,A,A,A,A,3,FRI 14:30-15:45 & TUE 14:30-15:45
138,COMM2012,WANG HAIYAN,1,A,A,A,A,7,FRI 11:30-12:45 & TUE 11:30-12:45
139,COMM2012,WANG HAIYAN,2,A,A,A,A,7,FRI 10:00-11:15 & TUE 10:00-11:15
179,CPED1001,CHAU HONG CHENG,20,A,A,A,A,4,MON 09:30-11:15
180,CPED1001,CHAU HONG CHENG,21,A,A,A,A,4,FRI 09:30-11:15
181,CPED1001,CHAU HONG CHENG,22,A,A,A,A,4,FRI 11:30-13:15
192,CPED1001,WONG SOI CHEONG,26,A,A,A,A,3,FRI 09:30-11:15


In [12]:
# 保存到CSV文件
sorted_df.to_csv('cm.csv', index=False)